# 1 Разработка модели машинного обучения

### Обучающая выборка данных разделена на обучающую и тестовую выборку

Выборки будут разделены в пропорции 80%/20% потому-что:

Если тестовая выборка слишком мала (например, 10% данных), оценка модели может быть ненадежной из-за высокой дисперсии. Это может привести к ошибочным выводам о качестве модели.
Если тестовая выборка слишком велика (например, 30% данных), модель может не получить достаточно данных для обучения, что приведет к недообучению.
Соотношение 80/20 обеспечивает баланс, минимизируя риск как переобучения, так и недообучения.


In [40]:
from sklearn.model_selection import train_test_split
import pandas as pd
import xgboost as xgb
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import numpy as np
import time
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor


In [41]:
full_df_path = "~/Загрузки/BIG DATA/df_2.csv"
data_y_path = "~/Загрузки/BIG DATA/target_2.csv"

In [42]:
full_df = pd.read_csv(full_df_path)

def drop_columns_with_lowest_correlation(df, n=10, columns_to_drop=None):
    """
    Удаляет столбцы с наименьшей корреляцией из DataFrame и возвращает его.

    Параметры:
    df (pd.DataFrame): Исходный DataFrame.
    n (int): Количество столбцов с наименьшей корреляцией, которые нужно удалить.

    Возвращает:
    pd.DataFrame: DataFrame с удаленными столбцами.
    """

    # Вычисляем матрицу корреляции
    corr_matrix = df.corr()
    
    # Преобразуем матрицу корреляции в одномерный Series, исключая диагональные элементы
    corr_series = corr_matrix.unstack().sort_values().drop_duplicates()
    
    # Исключаем корреляции с самими собой (значения 1.0)
    corr_series = corr_series[corr_series != 1.0]

    # Получаем n пар столбцов с наименьшей корреляцией
    low_corr_pairs = corr_series.head(n).index.tolist()

    # Собираем уникальные столбцы для удаления
    if columns_to_drop is None:
        columns_to_drop = set()
        for pair in low_corr_pairs:
            columns_to_drop.add(pair[0])
            columns_to_drop.add(pair[1])

    # Удаляем столбцы из DataFrame
    df_dropped = df.drop(columns=columns_to_drop)

    return df_dropped, columns_to_drop

full_df, columns_to_drop = drop_columns_with_lowest_correlation(full_df)
print(columns_to_drop)

{'26', '5', '27', '6', '12', '3', '19', '25', '16', '2', '17', '7', '0', '11', '21', '14', '10'}


In [43]:
# Так как ресурсы ограничены, то уменьшим размер данных до 100.000 строк
# Таким образом, мы получим предположительные значения метрик моделей и их скорости обучения
full_df = full_df.head(100_000).copy()

In [44]:
data_y = pd.read_csv(data_y_path)
data_y = data_y.head(int(len(full_df))).copy()

In [45]:
data_y = data_y.drop(["0"], axis=1)
data_y.head()

,1
0,5092.295833
1,2310.743036
2,3887.040957
3,3437.476122
4,4166.549638


In [46]:
# Разделение данных на обучающую и тестовую выборки в соотношении 80/20
X_train, X_test = train_test_split(full_df, test_size=0.2, random_state=42)

In [47]:
y_train, y_test = train_test_split(data_y, test_size=0.2, random_state=42)

In [48]:
X_train.to_csv("./X_train.csv")
X_test.to_csv("./X_test.csv")
y_train.to_csv("./y_train.csv")
y_test.to_csv("./y_test.csv")

X_val = pd.read_csv("~/Загрузки/BIG DATA/df_val.csv")
X_val, columns_to_drop = drop_columns_with_lowest_correlation(X_val, columns_to_drop=columns_to_drop)
X_val = X_val.head(100_000).copy()
X_val.to_csv("./X_val.csv")


y_val = pd.read_csv("~/Загрузки/BIG DATA/target_val.csv")
y_val = y_val.head(int(len(X_val))).copy()
y_val = y_val.drop(["0"], axis=1)

In [49]:
y_val.to_csv("./y_val.csv")

### Испытано как минимум три алгоритма разной сущности (RandomForest и XGBoost – разные сущности, а XGBoost и GradientBoosting – нет). Для оценки точности следует использовать тестовую выборку

Выбранные алгоритмы - XGBoost, RandomForest, LinearRegression


* LinearRegression позволяет оценить, насколько данные подходят для линейной модели. Если линейная зависимость присутствует, этот метод может дать хорошие результаты с минимальными вычислительными затратами.
* RandomForest и XGBoost позволяют улавливать нелинейные зависимости и сложные взаимодействия между признаками, что особенно полезно для задач регрессии с высокой сложностью данных.


### XGBoost

In [50]:
def xgboost_test(X_train, y_train, X_test, y_test):
    """
    Обучает модель XGBoost для задачи регрессии и возвращает модель, предсказания, метрики и время выполнения.

    Параметры:
    - X_train: Признаки обучающей выборки.
    - y_train: Целевая переменная обучающей выборки.
    - X_test: Признаки тестовой выборки.
    - y_test: Целевая переменная тестовой выборки.

    Возвращает:
    - model: Обученная модель XGBoost.
    - y_pred: Предсказания на тестовой выборке.
    - metrics: Словарь с метриками (R², RMSE, MSE, MAE).
    - execution_time: Словарь с временем выполнения (обучение, предсказание).
    """
    # Преобразование данных в формат DMatrix (оптимизированный для XGBoost)
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)

    # Параметры модели
    params = {
        'objective': 'reg:squarederror',  # Задача регрессии
        'eval_metric': 'rmse',           # Метрика оценки (среднеквадратичная ошибка)
        'max_depth': 6,                  # Максимальная глубина дерева
        'eta': 0.5,                      # Скорость обучения
        'subsample': 0.8,                 # Доля данных для обучения каждого дерева
        'colsample_bytree': 0.8,          # Доля признаков для обучения каждого дерева
        'seed': 42                        # Фиксация случайности
    }

    # Измерение времени обучения модели
    start_train = time.time()
    num_rounds = 10  # Количество итераций (деревьев)
    model = xgb.train(params, dtrain, num_rounds)
    end_train = time.time()
    train_time = end_train - start_train

    # Измерение времени предсказания
    start_predict = time.time()
    y_pred = model.predict(dtest)
    end_predict = time.time()
    predict_time = end_predict - start_predict

    # Вычисление метрик
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)  # RMSE
    r2 = r2_score(y_test, y_pred)  # R²
    mae = mean_absolute_error(y_test, y_pred)  # MAE

    # Вывод метрик
    print(f"Среднеквадратичная ошибка (MSE): {mse}")
    print(f"Среднеквадратичная ошибка (RMSE): {rmse}")
    print(f"Коэффициент детерминации (R²): {r2}")
    print(f"Точность (MAE): {mae}")

    # Вывод времени выполнения
    print(f"Время обучения модели: {train_time:.2f} секунд")
    print(f"Время предсказания: {predict_time:.2f} секунд")

    # Возвращаем модель, предсказания, метрики и время выполнения
    metrics = {'R²': r2, 'RMSE': rmse, 'MSE': mse, 'MAE': mae}
    execution_time = {'train_time': train_time, 'predict_time': predict_time}
    return model, y_pred, metrics, execution_time

In [51]:
xgboost_results = xgboost_test(X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test)

Среднеквадратичная ошибка (MSE): 1128926.875
Среднеквадратичная ошибка (RMSE): 1062.509705838022
Коэффициент детерминации (R²): 0.3226301670074463
Точность (MAE): 864.2380981445312
Время обучения модели: 0.31 секунд
Время предсказания: 0.01 секунд


In [52]:
import pickle

In [53]:
def save_model(model):
    with open('xgboost_model.pkl', 'wb') as model_file:
        pickle.dump(model, model_file)

save_model(xgboost_results[0])

### RandomForest

In [54]:
def random_forest_test(X_train, y_train, X_test, y_test):
    """
    Обучает модель Random Forest для задачи регрессии и возвращает модель, предсказания, метрики и время выполнения.

    Параметры:
    - X_train: Признаки обучающей выборки.
    - y_train: Целевая переменная обучающей выборки.
    - X_test: Признаки тестовой выборки.
    - y_test: Целевая переменная тестовой выборки.

    Возвращает:
    - model: Обученная модель RandomForestRegressor.
    - y_pred: Предсказания на тестовой выборке.
    - metrics: Словарь с метриками (R², RMSE, MSE, MAE).
    - execution_time: Словарь с временем выполнения (обучение, предсказание).
    """
    # Параметры модели
    params = {
        'n_estimators': 10,  # Количество деревьев
        'max_depth': 6,  # Максимальная глубина дерева
        'min_samples_split': 2,  # Минимальное количество образцов для разделения узла
        'min_samples_leaf': 1,  # Минимальное количество образцов в листе
        'random_state': 42  # Фиксация случайности
    }

    # Создание и обучение модели
    model = RandomForestRegressor(**params)

    # Измерение времени обучения модели
    start_train = time.time()
    model.fit(X_train, y_train)
    end_train = time.time()
    train_time = end_train - start_train

    # Измерение времени предсказания
    start_predict = time.time()
    y_pred = model.predict(X_test)
    end_predict = time.time()
    predict_time = end_predict - start_predict

    # Вычисление метрик
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)  # RMSE
    r2 = r2_score(y_test, y_pred)  # R²
    mae = mean_absolute_error(y_test, y_pred)  # MAE

    # Вывод метрик
    print(f"Среднеквадратичная ошибка (MSE): {mse}")
    print(f"Среднеквадратичная ошибка (RMSE): {rmse}")
    print(f"Коэффициент детерминации (R²): {r2}")
    print(f"Средняя абсолютная ошибка (MAE): {mae}")

    # Вывод времени выполнения
    print(f"Время обучения модели: {train_time:.2f} секунд")
    print(f"Время предсказания: {predict_time:.2f} секунд")

    # Возвращаем модель, предсказания, метрики и время выполнения
    metrics = {'R²': r2, 'RMSE': rmse, 'MSE': mse, 'MAE': mae}
    execution_time = {'train_time': train_time, 'predict_time': predict_time}
    return model, y_pred, metrics, execution_time

In [55]:
rf_results = random_forest_test(X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test)

/home/c2/main_venv/lib/python3.11/site-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Среднеквадратичная ошибка (MSE): 1134000.6474228955
Среднеквадратичная ошибка (RMSE): 1064.8946649424513
Коэффициент детерминации (R²): 0.31958579584265046
Средняя абсолютная ошибка (MAE): 866.9995806651078
Время обучения модели: 4.37 секунд
Время предсказания: 0.01 секунд


In [56]:
def save_model(model):
    with open('rf_model.pkl', 'wb') as model_file:
        pickle.dump(model, model_file)

save_model(rf_results[0])

### Linear Regression

In [57]:
def linear_regression_test(X_train, y_train, X_test, y_test):
    """
    Обучает модель Linear Regression для задачи регрессии и возвращает модель, предсказания, метрики и время выполнения.

    Параметры:
    - X_train: Признаки обучающей выборки.
    - y_train: Целевая переменная обучающей выборки.
    - X_test: Признаки тестовой выборки.
    - y_test: Целевая переменная тестовой выборки.

    Возвращает:
    - model: Обученная модель LinearRegression.
    - y_pred: Предсказания на тестовой выборке.
    - metrics: Словарь с метриками (R², RMSE, MSE, MAE).
    - execution_time: Словарь с временем выполнения (обучение, предсказание).
    """
    # Создание и обучение модели
    model = LinearRegression()

    # Измерение времени обучения модели
    start_train = time.time()
    model.fit(X_train, y_train)
    end_train = time.time()
    train_time = end_train - start_train

    # Измерение времени предсказания
    start_predict = time.time()
    y_pred = model.predict(X_test)
    end_predict = time.time()
    predict_time = end_predict - start_predict

    # Вычисление метрик
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)  # RMSE
    r2 = r2_score(y_test, y_pred)  # R²
    mae = mean_absolute_error(y_test, y_pred)  # MAE

    # Вывод метрик
    print(f"Среднеквадратичная ошибка (MSE): {mse}")
    print(f"Среднеквадратичная ошибка (RMSE): {rmse}")
    print(f"Коэффициент детерминации (R²): {r2}")
    print(f"Средняя абсолютная ошибка (MAE): {mae}")

    # Вывод времени выполнения
    print(f"Время обучения модели: {train_time:.2f} секунд")
    print(f"Время предсказания: {predict_time:.2f} секунд")

    # Возвращаем модель, предсказания, метрики и время выполнения
    metrics = {'R²': r2, 'RMSE': rmse, 'MSE': mse, 'MAE': mae}
    execution_time = {'train_time': train_time, 'predict_time': predict_time}
    return model, y_pred, metrics, execution_time

In [58]:
lr_results = linear_regression_test(X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test)

Среднеквадратичная ошибка (MSE): 1165439.6021936312
Среднеквадратичная ошибка (RMSE): 1079.5552798229608
Коэффициент детерминации (R²): 0.3007220399546068
Средняя абсолютная ошибка (MAE): 878.3796678118957
Время обучения модели: 0.04 секунд
Время предсказания: 0.00 секунд


In [59]:
def save_model(model):
    with open('lr_model.pkl', 'wb') as model_file:
        pickle.dump(model, model_file)

save_model(lr_results[0])

### Испытанные алгоритмы сравнены по точности, а также по скорости работы

Значения времени выполнения, записанные в переменные во время тестирования, преведены ниже

##### XGBoost

    Среднеквадратичная ошибка (MSE): 1128926.875
    Среднеквадратичная ошибка (RMSE): 1062.509705838022
    Коэффициент детерминации (R²): 0.3226301670074463
    Точность (MAE): 864.2380981445312
    Время обучения модели: 0.25 секунд
    Время предсказания: 0.00 секунд

##### RandomForest

    Среднеквадратичная ошибка (MSE): 1134000.6474228955
    Среднеквадратичная ошибка (RMSE): 1064.8946649424513
    Коэффициент детерминации (R²): 0.31958579584265046
    Средняя абсолютная ошибка (MAE): 866.9995806651078
    Время обучения модели: 4.46 секунд
    Время предсказания: 0.01 секунд

##### LinearRegression

    Среднеквадратичная ошибка (MSE): 1165439.6021936312 
    Среднеквадратичная ошибка (RMSE): 1079.5552798229608
    Коэффициент детерминации (R²): 0.3007220399546068
    Средняя абсолютная ошибка (MAE): 878.3796678118957
    Время обучения модели: 0.03 секунд
    Время предсказания: 0.00 секунд

Таким образом,
* Самая быстрая модель: LinearRegression
* Самый лучший R2: LinearRegression
* Самый лучший RMSE: LinearRegression

Модель будет немного улучшена и добавлена в pipeline

### Лучший алгоритм реализован в виде пайплайна, принимающего на вход пути к файлам, а на выходе возвращает обученную модель, а также метрики модели на валидационной выборке в виде словаря (необходимые метрики: средний коэффициент детерминации R2 и средний RMSE).

Реализация пайплайна для алгоритма LinearRegression с использованием функции для его тестирования:

In [60]:
def pipeline(data_X_path: str, data_y_path: str, data_x_valid_path: str, data_y_valid_path: str):
    data_X = pd.read_csv(data_X_path)
    data_X, columns_to_drop = drop_columns_with_lowest_correlation(data_X, columns_to_drop=columns_to_drop)
    data_X = data_X.head(100_000).copy()
    data_y = pd.read_csv(data_y_path)
    data_y = data_y.head(int(len(data_X))).copy()
    data_y = data_y.drop(["0"], axis=1)
    data_y.head()
    X_train, X_test = train_test_split(full_df, test_size=0.2, random_state=42)
    y_train, y_test = train_test_split(data_y, test_size=0.2, random_state=42)
    results = linear_regression_test(X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test)

    X_valid = pd.read_csv(data_x_valid_path)
    y_valid = pd.read_csv(data_y_valid_path)

    model = results[0]
    y_pred = model.predict(X_valid)

    mse = mean_squared_error(y_valid, y_pred)
    rmse = np.sqrt(mse)  # RMSE
    r2 = r2_score(y_valid, y_pred)  # R²

    print(f"Среднеквадратичная ошибка (RMSE): {rmse}")
    print(f"Коэффициент детерминации (R²): {r2}")

    metrics = {'R²': r2, 'RMSE': rmse}

    return model, metrics

In [61]:
full_df_path = "~/Загрузки/BIG DATA/df_2.csv"
data_y_path = "~/Загрузки/BIG DATA/target_2.csv"
data_x_val_path = "./X_val.csv"
data_y_val_path = "./y_val.csv"

model, metrics = pipeline(data_y_path=data_y_path, data_X_path=full_df_path, data_x_valid_path=data_x_val_path, data_y_valid_path=data_y_val_path)


ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [ ]:
from datetime import datetime

def save_metrics(metrics):
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    with open('~/Рабочий стол/C2_M5/metrics.txt', 'w') as metrics_file:
        metrics_file.write(f"Timestamp: {timestamp}\n")
        for key, value in metrics.items():
            metrics_file.write(f"{key}: {value}\n")

save_metrics(metrics)